<a href="https://colab.research.google.com/github/jkraybill/gpt-2-b/blob/finetuning/GPT2-b-finetuning2-774M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To try out GPT-2, do this:

- go to the "Runtime" menu and click "Change runtime type" and make sure this is a Python 3 notebook, running with GPU hardware acceleration.
- use the "Files" section to the left to upload a text file called "train.txt" which contains all the text you want to train on, and  a text file called "val.txt" which contains all the text you want to validate on.
- run the steps below in order.

In [0]:
import os
import json
import random
import re

In [2]:
!git clone https://github.com/jkraybill/gpt-2-b.git

Cloning into 'gpt-2-b'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 301 (delta 160), reused 301 (delta 160), pack-reused 0
Receiving objects: 100% (301/301), 4.40 MiB | 3.94 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [3]:
cd gpt-2-b

/content/gpt-2-b


In [13]:
!pip3 install -r requirements.txt

In [8]:
!python ./download_model.py 774M

Fetching checkpoint: 1.00kit [00:00, 725kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 37.4Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 621kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:59, 52.0Mit/s]                                 
Fetching model.ckpt.index: 16.0kit [00:00, 6.95Mit/s]                                               
Fetching model.ckpt.meta: 1.38Mit [00:00, 36.1Mit/s]                                                
Fetching vocab.bpe: 457kit [00:00, 28.0Mit/s]                                                       


The below step encodes your corpus into "NPZ" tokenized format for GPT-2.



In [11]:
!PYTHONPATH=src ./encode.py ../corpus-j2-train.txt train.txt.npz --model_name 774M
!PYTHONPATH=src ./encode.py ../corpus-j2-val.txt val.txt.npz --model_name 774M

Reading files
100% 1/1 [00:38<00:00, 38.42s/it]
Writing train.txt.npz
Reading files
100% 1/1 [00:06<00:00,  6.91s/it]
Writing val.txt.npz


Training is below. It will auto-stop after val loss stops going down for a while, but you may want to interrupt it early.

"sample_every" controls how often you get sample output from the trained model.

"save_every" controls how often the model is saved.

"learning_rate" is the AI learning rate. 0.00005 is the rate I've gotten the best results with, but I think most people are running with significantly higher rates, so you could try adjusting it.

"layers_to_train" controls the number of layers to fine-tune. Reduce this number if you are getting Out of Memory type errors. I hit them in Colab at values larger than 336 for the 774M model.


In [29]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/jkraybill/gpt-2-b
   f3284d2..f4bf21b  finetuning -> origin/finetuning
Updating f3284d2..f4bf21b
Fast-forward
 src/model.py | 3 +--
 1 file changed, 1 insertion(+), 2 deletions(-)


In [0]:
!PYTHONPATH=src ./train.py --dataset train.txt.npz --val_dataset val.txt.npz --sample_every=1000 --save_every=25 --learning_rate=0.00005 --model_name=774M --layers_to_train=336 --optimizer=adafactor






2019-09-24 14:30:11.425453: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2019-09-24 14:30:11.425713: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x27a3100 executing computations on platform Host. Devices:
2019-09-24 14:30:11.425750: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-24 14:30:11.427746: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-09-24 14:30:11.502314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-09-24 14:30:11.503132: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x27a32c0 executing computations on platform CUDA. Devices:
2019-09-24 14:30:11.503166: I tensorflow/compiler/xla/service/service.cc:175]   Stream

The step below simply copies your trained model to the model directory, so the output will use your training. If you don't do this, you will be running against the trained GPT-2 model without your finetuning training.

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/774M/

Run the below step to generate unconditional samples (i.e. "dream mode").

"top_k" controls how many options to consider per word (the larger, the more "diverse" the output - anything from 1 to about 50 usually works, I think values around 10 are pretty good).

"temperature" controls the sampling of the words, from 0 to 1 where 1 is the most "random".

"length" controls the number of words in each sample output.

This command will run continuously until you turn it off.

In [0]:
!python3 src/generate_unconditional_samples.py --top_k 20 --temperature 0.8 --length=300 --model_name=774M

Run the command below to run in interactive / "completion" mode. You will get a prompt; just type in whatever prompt text you want, and the model will attempt to complete it "nsamples" times.

"top_k", "length", and "temperature" work as specified above.

In [0]:
!python3 src/interactive_conditional_samples.py --top_k 1 --length=30 --temperature 0.1 --nsamples 3 --model_name=774M